__/Q1: Some of the facilities charge a fee to members, but some do not. Write a SQL query to produce a list of the names of the facilities that do.__

SELECT name, membercost
FROM Facilities
WHERE membercost != 0


__/* Q2: How many facilities do not charge a fee to members? */__

SELECT count(membercost)
FROM Facilities
WHERE membercost != 0



__Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.__

SELECT facid, name, membercost, monthlymaintenance
FROM Facilities
WHERE membercost != 0 AND membercost < (.2 * monthlymaintenance)

__Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator.__


SELECT *
FROM Facilities
WHERE facid in (1, 5)


__Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question.__

SELECT name, monthlymaintenance,
CASE
WHEN monthlymaintenance > 100 THEN 'Expensive'
WHEN monthlymaintenance <= 100 THEN 'Cheap'
END AS label
FROM Facilities




__Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution.__

SELECT firstname, surname, joindate
FROM Members
WHERE joindate IN
(SELECT max(joindate)
FROM Members)

__Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.__

SELECT DISTINCT(CONCAT(Members.firstname, " ", Members.surname)) AS membername, Facilities.name
FROM Members
INNER JOIN Bookings
ON Members.memid = Bookings.memid
INNER JOIN Facilities
ON Bookings.facid = Facilities.facid
WHERE Bookings.facid in (0, 1)
ORDER BY membername


__Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.__


Solution:
SELECT Facilities.name, CONCAT(Members.firstname, " ", Members.surname) AS membername,
CASE
WHEN Bookings.memid > 0 then guestcost
ELSE membercost
END AS booking_cost
FROM Bookings
INNER JOIN
Facilities ON Bookings.facid = Facilities.facid
INNER JOIN
Members ON Members.memid = Bookings.memid
WHERE Bookings.starttime LIKE '2012-09-14%' AND 
CASE
WHEN Bookings.memid > 0 then guestcost
ELSE membercost
END > 30
ORDER BY booking_cost DESC


__Q9: This time, produce the same result as in Q8, but using a subquery.__

SELECT Facilities.name, CONCAT(Members.firstname, " ", Members.surname) AS membername
FROM Bookings
INNER JOIN
Facilities ON Bookings.facid = Facilities.facid
INNER JOIN
Members ON Members.memid = Bookings.memid
WHERE Members.memid IN 

(SELECT memid,
CASE
WHEN Bookings.memid > 0 then guestcost
ELSE membercost
END AS booking_cost
FROM Bookings
INNER JOIN
Facilities ON Bookings.facid = Facilities.facid
WHERE Bookings.starttime LIKE '2012-09-14%'
AND 
CASE
WHEN Bookings.memid > 0 then guestcost
ELSE membercost
END > 30)

__**Note: I was not able to figure this out.  It still returns an error (#1241 - Operand should contain 1 column(s))__

/* PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  

In [30]:
import sqlite3
import pandas as pd

connection = sqlite3.connect('sqlite_db_pythonsqlite.db')
cur = connection.cursor()

In [31]:
cur.execute('SELECT * FROM facilities') 
data = pd.DataFrame(cur.fetchall())
data.columns = [x[0] for x in cur.description]
data

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


__Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!__


In [1]:
cur.execute('SELECT Facilities.name, SUM(booking_cost) AS TotalRevenue,
CASE
WHEN Bookings.memid > 0 then guestcost
ELSE membercost
END AS booking_cost
FROM Bookings
INNER JOIN
Facilities ON Bookings.facid = Facilities.facid
WHERE
CASE
WHEN Bookings.memid > 0 then guestcost
ELSE membercost
END
WHERE TotalRevenue < 1000
GROUP BY Facilities.name') 

data = pd.DataFrame(cur.fetchall())
data.columns = [x[0] for x in cur.description]
data


# Here is my best guess - struggling wiht the abilty to use the TotalRevenue column



SyntaxError: EOL while scanning string literal (654403481.py, line 1)

In [33]:
# just the SQL:
# SELECT Facilities.name, SUM(booking_cost) AS TotalRevenue,
# CASE
# WHEN Bookings.memid > 0 then guestcost
# ELSE membercost
# END AS booking_cost
# FROM Bookings
# INNER JOIN
# Facilities ON Bookings.facid = Facilities.facid
# WHERE
# CASE
# WHEN Bookings.memid > 0 then guestcost
# ELSE membercost
# END
# WHERE TotalRevenue < 1000
# GROUP BY Facilities.name

__Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order__

In [34]:
cur.execute('SELECT m.surname, m.firstname, m.recommendedby AS recommender_id, r.surname as r_surname, r.firstname as r_firstname FROM Members AS m LEFT JOIN Members AS r ON m.recommendedby = r.memid WHERE m.recommendedby !=0 ORDER BY m.surname, m.firstname') 

data = pd.DataFrame(cur.fetchall())
data.columns = [x[0] for x in cur.description]
data

,surname,firstname,recommender_id,r_surname,r_firstname
0,Bader,Florence,9,Stibbons,Ponder
1,Baker,Anne,9,Stibbons,Ponder
2,Baker,Timothy,13,Farrell,Jemima
3,Boothe,Tim,3,Rownam,Tim
4,Butters,Gerald,1,Smith,Darren
5,Coplin,Joan,16,Baker,Timothy
6,Crumpet,Erica,2,Smith,Tracy
7,Dare,Nancy,4,Joplette,Janice
8,Farrell,David,,None,None
9,Farrell,Jemima,,None,None


In [40]:
# just the SQL:
# SELECT m.surname, m.firstname, m.recommendedby AS recommender_id, r.surname as r_surname, r.firstname as r_firstname
# FROM Members AS m
# LEFT JOIN Members AS r
# ON m.recommendedby = r.memid
# WHERE m.recommendedby !=0
# ORDER BY m.surname, m.firstname

__Q12: Find the facilities with their usage by member, but not guests__

In [36]:
cur.execute('SELECT memid, facid, count(bookid) as times_used FROM Bookings WHERE memid > 0 GROUP BY memid, facid ORDER BY memid ASC, times_used DESC') 

data = pd.DataFrame(cur.fetchall())
data.columns = [x[0] for x in cur.description]
data

,memid,facid,times_used
0,1,2,132
1,1,8,28
2,1,4,28
3,1,3,28
4,1,1,19
...,...,...,...
197,35,3,1
198,36,4,2
199,36,3,2
200,36,2,2


In [37]:
# just the SQL
# SELECT memid, facid, count(bookid) as times_used
# FROM Bookings
# WHERE memid > 0
# GROUP BY memid, facid
# ORDER BY memid ASC, times_used DESC

__Q13: Find the facilities usage by month, but not guests__

In [38]:
cur.execute('SELECT MONTH(starttime) AS months, facid, count(bookid) as times_used FROM Bookings WHERE memid > 0 GROUP BY months, facid ORDER BY months ASC, times_used DESC') 

data = pd.DataFrame(cur.fetchall())
data.columns = [x[0] for x in cur.description]
data

OperationalError: no such function: MONTH

In [41]:
# just the sql
# SELECT MONTH(starttime) AS months, facid, count(bookid) as times_used
# FROM Bookings
# WHERE memid > 0
# GROUP BY months, facid
# ORDER BY months ASC, times_used DESC